In [1]:
from bs4 import BeautifulSoup as bs
from numpy import nan
import seaborn as sns
import pandas as pd
import requests
from time import sleep
import matplotlib.pyplot as plt
import warnings
import folium
import datetime

#그래프 한글 오류 해결
from matplotlib import font_manager, rc

font_path = 'malgun.ttf'
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)

#경고 메시지 무시
warnings.filterwarnings("ignore")

#마이너스 부호 유니코드
plt.style.use('ggplot')
plt.rcParams['axes.unicode_minus']=False

In [2]:
titanic = sns.load_dataset('titanic')
titanic.head(2)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False


In [13]:
df = titanic.loc[:,['age','sex','class','fare','survived']]
df.head(3)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1


In [18]:
list(df.groupby('sex')) #해당 열의 값들을 종목화하여 만든다. 리스트 함수를 이용하여로 리스트로 만들수있다

# numeric_only = False(기본값) : True할 시 수치를 가진 값들만 계산하여 평균값을 도출한다.
grouped = df.groupby('class')
grouped.mean(numeric_only = True)

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [16]:
for key, group in grouped:    #for문 변수 2개 입력방법
    print('*key:',key)
    print('*number:',len(group))
    print(group.head(3))

*key: First
*number: 216
    age     sex  class     fare  survived
1  38.0  female  First  71.2833         1
3  35.0  female  First  53.1000         1
6  54.0    male  First  51.8625         0
*key: Second
*number: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
*key: Third
*number: 491
    age     sex  class   fare  survived
0  22.0    male  Third  7.250         0
2  26.0  female  Third  7.925         1
4  35.0    male  Third  8.050         0


In [20]:
mask1 = df['class'] == 'First'
mask2 = df['class'] == 'Second'
mask3 = df['class'] == 'Third'

df1 = df[mask1].mean(numeric_only = True)
df2 = df[mask2].mean(numeric_only = True)
df3 = df[mask3].mean(numeric_only = True)

pd.DataFrame({'First':df1, 'Second':df2, 'Third':df3})
pd.DataFrame([df1,df2,df3], index = ['First','Second','Third'])

,First,Second,Third
age,38.233441,29.877630,25.140620
fare,84.154687,20.662183,13.675550
survived,0.629630,0.472826,0.242363


In [22]:
grouped2 = df.groupby(['class','sex'])
for key, group in grouped2:
    print('*key:',key)
    print('*number:',len(group))
    print(group.head(3))

*key: ('First', 'female')
*number: 94
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
11  58.0  female  First  26.5500         1
*key: ('First', 'male')
*number: 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
*key: ('Second', 'female')
*number: 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
*key: ('Second', 'male')
*number: 108
     age   sex   class  fare  survived
17   NaN  male  Second  13.0         1
20  35.0  male  Second  26.0         0
21  34.0  male  Second  13.0         1
*key: ('Third', 'female')
*number: 144
     age     sex  class     fare  survived
2   26.0  female  Third   7.9250         1
8   27.0  female  Third  11.1333         1
10   4.0

In [24]:
grouped2.mean()

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [26]:
grouped = df.groupby(['class'])
grouped.std(numeric_only=True)  #std 표준편차 집계

,age,fare,survived
class,,,
First,14.802856,78.380373,0.484026
Second,14.001077,13.417399,0.500623
Third,12.495398,11.778142,0.428949


In [27]:
grouped[['age','fare']].std()

,age,fare
class,,
First,14.802856,78.380373
Second,14.001077,13.417399
Third,12.495398,11.778142


In [35]:
def min_max(x):
    return x.max() - x.min()

grouped[['age','fare']].agg(min_max)  
#agg 함수 : 여러개의 함수를 제공해준다.
# agg({'컬럼':'함수', '컬럼':'함수'})
# agg({'컬럼':[함수1, 함수2], '컬럼':[함수1, 함수2]})

,age,fare
class,,
First,79.08,512.3292
Second,69.33,73.5000
Third,73.58,69.5500


In [36]:
grouped[['age','fare']].agg([max,min,min_max])

age                    fare               
         max   min min_max       max  min   min_max
class                                              
First   80.0  0.92   79.08  512.3292  0.0  512.3292
Second  70.0  0.67   69.33   73.5000  0.0   73.5000
Third   74.0  0.42   73.58   69.5500  0.0   69.5500

In [38]:
grouped[['age','fare']].agg({'age':['mean','std'],'fare':['min','max']})

age            fare          
             mean        std  min       max
class                                      
First   38.233441  14.802856  0.0  512.3292
Second  29.877630  14.001077  0.0   73.5000
Third   25.140620  12.495398  0.0   69.5500

In [49]:
grouped = df.groupby('class')
for key, group in grouped:
    print(key, len(group))

    
#그룹핑객체.filter(조건함수) : 조건에 부합되는 데이터 프레임들을 결합하여 1개의 데이터 프레임

grouped_filter = grouped.filter(lambda df: len(df) > 200)
grouped_filter

First 216
Second 184
Third 491


,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
...,...,...,...,...,...
885,39.0,female,Third,29.1250,0
887,19.0,female,First,30.0000,1
888,NaN,female,Third,23.4500,0
889,26.0,male,First,30.0000,1


In [50]:
#filter(함수)는 데이터프레임 객체를 전달받아 True / False로 반환해야한다.
def over_200(df):
    return len(df) > 200

grouped.filter(over_200)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
...,...,...,...,...,...
885,39.0,female,Third,29.1250,0
887,19.0,female,First,30.0000,1
888,NaN,female,Third,23.4500,0
889,26.0,male,First,30.0000,1
